# Optimize Initial Conditions
## Yabox

In [1]:
from scipy.optimize import brute
import numpy as np
from datetime import datetime,timedelta
import pandas as pd
from yabox import DE

# Load solver

In [2]:
LoadBH=True
if LoadBH:
    import LearnerICRayNoLoadBH as L 
else:
    import LearnerICRayNoLoad as L 

# Load Data

In [3]:
def load_confirmed(state, start_date):
    dateparse = lambda x: datetime.strptime(x, '%Y-%m-%d')
    df = pd.read_csv('./data/confirmados.csv',delimiter=',',parse_dates=True, date_parser=dateparse)
    y=[]
    x=[]
    for i in range(0,len(df.date)):
        y.append(df[state].values[i])
        x.append(df.date.values[i])
    df2=pd.DataFrame(data=y,index=x,columns=[""])
    df2=df2[start_date:]
    del x,y,df,dateparse
    return df2

def load_dead(state, start_date):
    dateparse = lambda x: datetime.strptime(x, '%Y-%m-%d')
    df = pd.read_csv('./data/mortes.csv',delimiter=',',parse_dates=True, date_parser=dateparse)
    y=[]
    x=[]
    for i in range(0,len(df.date)):
        y.append(df[state].values[i])
        x.append(df.date.values[i])
    df2=pd.DataFrame(data=y,index=x,columns=[""])
    df2=df2[start_date:]
    del x,y,df,dateparse
    return df2

# Data for States

In [4]:
dfparam = pd.read_csv("data/param.csv")
dfparam

,state,start-date,prediction-range,s0,e0,a0,i0,r0,d0,START,RATIO,WCASES,WREC
0,SP,2020-03-15,200,8000000.0,0,0,800,300,250,100,0.15,0.55,0.01
1,ES,2020-04-01,200,475000.0,0,0,50,250,50,100,0.15,0.65,0.01
2,MG,2020-04-01,200,1100000.0,0,0,200,250,40,100,0.15,0.55,0.01
3,RJ,2020-03-20,200,700000.0,0,0,800,250,50,100,0.15,0.50,0.01
4,CE,2020-03-20,200,800000.0,0,0,800,250,50,100,0.15,0.50,0.01
5,PE,2020-03-20,200,700000.0,0,0,800,250,100,100,0.15,0.50,0.01
6,AM,2020-03-20,200,700000.0,0,0,800,250,100,100,0.15,0.50,0.01


In [5]:
df = pd.read_csv("data/dados_total_estados.csv")
df

,index,date,state,city,place_type,confirmed,deaths,order_for_place,is_last,popEst,city_ibge_code,confirmed_per_100k_inhabitants,death_rate
0,0,2020-06-21,AP,TOTAL,state,21840,359,93,True,845731.0,16.0,2582.38140,0.0164
1,1,2020-06-20,AP,TOTAL,state,21574,353,92,False,845731.0,16.0,2550.92931,0.0164
2,2,2020-06-19,AP,TOTAL,state,19922,348,91,False,845731.0,16.0,2355.59534,0.0175
3,3,2020-06-18,AP,TOTAL,state,19387,342,90,False,845731.0,16.0,2292.33645,0.0176
4,4,2020-06-17,AP,TOTAL,state,18618,338,89,False,845731.0,16.0,2201.40920,0.0182
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2652,208291,2020-02-29,SP,TOTAL,state,2,0,5,False,45919049.0,35.0,0.00436,0.0000
2653,208292,2020-02-28,SP,TOTAL,state,2,0,4,False,45919049.0,35.0,0.00436,0.0000
2654,208293,2020-02-27,SP,TOTAL,state,1,0,3,False,45919049.0,35.0,0.00218,0.0000
2655,208294,2020-02-26,SP,TOTAL,state,1,0,2,False,45919049.0,35.0,0.00218,0.0000


# Functions for Optimization

In [6]:
from scipy.integrate import odeint
import os
import sys
import io
import ray
import gc

def create_f(state,e0,a0,r0,d0,date,startNCases, ratio, predict_range, version):
                
    def fobjective(point):
        
        dead=  load_dead(state,date)
        data = load_confirmed(state,date)*(1-ratio)-dead
        
        cleanRecovered=False
        s0, deltaDate, i0, weigthCases, weigthRecov = point
        end_date=datetime.strptime(date, "%Y-%m-%d") + timedelta(days=+int(deltaDate))
                    
        f=L.Learner.remote(state, end_date.strftime("%Y-%m-%d"), predict_range,\
                           s0, e0, a0, i0, r0, d0, startNCases, ratio, weigthCases, weigthRecov, \
                           cleanRecovered, version, data, dead, savedata=False)
        result = f.train.remote() 
        result = ray.get(result) 

        del end_date,cleanRecovered, data, dead, point,f         

        gc.collect()

        return result
    return fobjective

In [7]:
@ray.remote(memory=50 * 1024 * 1024, max_calls=7)
def opt(state,s0,i0,e0,a0,r0,d0,date,startNCases, ratio, predict_range, version):

    bounds=[(0.8*s0,1.2*s0),(-10,10),(0.8*i0,1.2*i0),\
              (0.3,0.7),(.001,.05)]
    maxiterations=300
    f=create_f(state,e0,a0,r0,d0,date,startNCases, ratio, predict_range, version)
    de = DE(f, bounds, maxiters=maxiterations)
    for step in de.geniterator():
        try:
            idx = step.best_idx
            norm_vector = step.population[idx]
            best_params = de.denormalize([norm_vector])
            del norm_vector, idx
        except:
            print("error in function evaluation")
    p=best_params[0]
    del f, bounds, data, dead
    
    return p

# Main Code

In [8]:
states=['SP','ES','MG','RJ','PE','CE','AM']
# states=['MG','RJ','PE','CE','AM']
allStates=True
version="103"
ray.shutdown()
GB=1024*1024*1024
MB=1024*1024
import cupy
ray.init(object_store_memory=500*MB,memory=2*GB,lru_evict=True,\
         driver_object_store_memory=50*MB,num_gpus=256,num_cpus=7)
gc.enable()

optimal=[]
if allStates:
    for state in states:
        #remove previous history file
        strFile='./results/history_'+state+version+'.csv'
        if os.path.isfile(strFile):
            os.remove(strFile)
        query = dfparam.query('state == "{}"'.format(state)).reset_index()
        parameters = np.array(query.iloc[:, 2:])[0]
        date,predict_range,s0,e0,a0,i0,r0,d0,startNCases,ratio,wcases,wrec = parameters
        optimal.append(opt.remote(state,s0,i0,e0,a0,r0,d0,date,startNCases, ratio, predict_range, version))        
else:
    state = "ES" 
    #remove previous history file
    strFile='./results/history_'+state+version+'.csv'
    if os.path.isfile(strFile):
        os.remove(strFile)
    query = dfparam.query('state == "{}"'.format(state)).reset_index()
    parameters = np.array(query.iloc[:, 2:])[0]
    start_date,predict_range,s0,e0,a0,i0,r0,d0,startNCases,ratio,wcases,wrec = parameters
    optimal.append(opt.remote(state,s0,i0,e0,a0,r0,d0,date,startNCases, ratio, predict_range, version))        

2020-06-22 17:12:56,795	INFO resource_spec.py:212 -- Starting Ray with 1.95 GiB memory available for workers and up to 0.49 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-06-22 17:12:57,133	INFO services.py:1170 -- View the Ray dashboard at localhost:8265


In [ ]:
optimal=ray.get(optimal)


(pid=25246) basinhopping step 0: f 2.57216e+08
(pid=25332) basinhopping step 0: f 6.15596e+08
(pid=25334) basinhopping step 0: f 2.17459e+08
(pid=25241) basinhopping step 0: f 1.8413e+09
(pid=25212) basinhopping step 0: f 6.32457e+07
(pid=25334) basinhopping step 1: f -2.06219e+11 trial_f -2.06219e+11 accepted 1  lowest_f -2.06219e+11
(pid=25334) found new global minimum on step 1 with function value -2.06219e+11
(pid=25334) /home/ats4i/anaconda3/envs/geo_env/lib/python3.6/site-packages/scipy/integrate/odepack.py:248: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
(pid=25334)   warnings.warn(warning_msg, ODEintWarning)
(pid=25334) basinhopping step 2: f -2.06219e+11 trial_f 2.17495e+08 accepted 0  lowest_f -2.06219e+11
(pid=25333) basinhopping step 0: f -7.76909e+11
(pid=25333) basinhopping step 1: f -7.76909e+11 trial_f 4.45281e+07 accepted 0  lowest_f -7.76909e+11
(pid=25334) warning: basinhopping: loc

(pid=25241) basinhopping step 8: f 1.83194e+09 trial_f 1.96587e+09 accepted 0  lowest_f 1.83194e+09
(pid=25332) basinhopping step 10: f -5.14245e+11 trial_f -4.59255e+11 accepted 0  lowest_f -5.14245e+11
(pid=25361) basinhopping step 0: f 3.34261e+08
(pid=25246) basinhopping step 8: f -6.99477e+11 trial_f 3.20064e+08 accepted 0  lowest_f -6.99477e+11
(pid=25334) basinhopping step 7: f -5.02006e+11 trial_f -4.15738e+11 accepted 0  lowest_f -5.02006e+11
(pid=25232) basinhopping step 8: f -3.18361e+11 trial_f -9.33731e+10 accepted 0  lowest_f -3.18361e+11
(pid=25333) basinhopping step 4: f -1.60914e+12 trial_f -6.41041e+11 accepted 0  lowest_f -1.60914e+12
(pid=25333) basinhopping step 5: f -1.60914e+12 trial_f 3.16538e+07 accepted 0  lowest_f -1.60914e+12
(pid=25334) basinhopping step 8: f -5.02006e+11 trial_f -2.11472e+11 accepted 0  lowest_f -5.02006e+11
(pid=25212) basinhopping step 10: f -5.28626e+11 trial_f 6.41043e+07 accepted 0  lowest_f -5.28626e+11
(pid=25241) basinhopping step 

(pid=25361) basinhopping step 9: f 1.7366e+08 trial_f 3.31787e+08 accepted 0  lowest_f 1.7366e+08
(pid=25373) basinhopping step 9: f 9.97721e+07 trial_f 1.25952e+09 accepted 0  lowest_f 9.97721e+07
(pid=25501) basinhopping step 0: f -3.859e+11
(pid=25329) warning: basinhopping: local minimization failure
(pid=25329) basinhopping step 7: f 8.73504e+08 trial_f 9.85782e+08 accepted 0  lowest_f 8.73504e+08
(pid=25331) basinhopping step 5: f -8.43355e+10 trial_f 4.54079e+08 accepted 0  lowest_f -8.43355e+10
(pid=25327) basinhopping step 5: f -1.70651e+11 trial_f -1.70651e+11 accepted 1  lowest_f -1.70651e+11
(pid=25327) found new global minimum on step 5 with function value -1.70651e+11
(pid=25326) basinhopping step 6: f -5.80902e+11 trial_f 2.5888e+08 accepted 0  lowest_f -5.80902e+11
(pid=25373) basinhopping step 10: f 9.97721e+07 trial_f 1.06128e+10 accepted 0  lowest_f 9.97721e+07
(pid=25514) basinhopping step 0: f 3.15108e+08
(pid=25326) basinhopping step 7: f -5.80902e+11 trial_f -4.5

(pid=25515) basinhopping step 6: f 5.56453e+07 trial_f 3.3336e+08 accepted 0  lowest_f 5.56453e+07
(pid=25553) basinhopping step 4: f -1.26791e+11 trial_f 8.39306e+07 accepted 0  lowest_f -1.26791e+11
(pid=25515) basinhopping step 7: f 5.56453e+07 trial_f 1.51899e+09 accepted 0  lowest_f 5.56453e+07
(pid=25540) warning: basinhopping: local minimization failure
(pid=25540) basinhopping step 6: f -3.06385e+11 trial_f 3.101e+08 accepted 0  lowest_f -3.06385e+11
(pid=25515) basinhopping step 8: f 5.56453e+07 trial_f 3.37218e+08 accepted 0  lowest_f 5.56453e+07
(pid=25579) basinhopping step 2: f 2.0763e+09 trial_f 2.0763e+09 accepted 1  lowest_f 2.0763e+09
(pid=25579) found new global minimum on step 2 with function value 2.0763e+09
(pid=25579) /home/ats4i/anaconda3/envs/geo_env/lib/python3.6/site-packages/scipy/integrate/odepack.py:248: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
(pid=25579)   warnings.wa

(pid=25579) basinhopping step 10: f -6.49999e+13 trial_f 7.26451e+09 accepted 0  lowest_f -6.49999e+13
(pid=25666) basinhopping step 0: f 2.60484e+08
(pid=25666) /home/ats4i/anaconda3/envs/geo_env/lib/python3.6/site-packages/scipy/integrate/odepack.py:248: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
(pid=25666)   warnings.warn(warning_msg, ODEintWarning)
(pid=25605) basinhopping step 6: f -6.31775e+11 trial_f 4.75264e+07 accepted 0  lowest_f -6.31775e+11
(pid=25606) basinhopping step 8: f -3.84525e+11 trial_f -3.84525e+11 accepted 1  lowest_f -3.84525e+11
(pid=25606) found new global minimum on step 8 with function value -3.84525e+11
(pid=25554) basinhopping step 10: f -2.14443e+11 trial_f 8.32561e+07 accepted 0  lowest_f -2.14443e+11
(pid=25679) basinhopping step 0: f -8.91384e+13
(pid=25679) /home/ats4i/anaconda3/envs/geo_env/lib/python3.6/site-packages/scipy/integrate/odepack.py:248: ODEintWarning:

...

(pid=81725) basinhopping step 8: f -3.01249e+11 trial_f 6.45638e+08 accepted 0  lowest_f -3.01249e+11
(pid=81765) basinhopping step 1: f -6.97807e+13 trial_f -6.97807e+13 accepted 1  lowest_f -6.97807e+13
(pid=81765) found new global minimum on step 1 with function value -6.97807e+13
(pid=81765) /home/ats4i/anaconda3/envs/geo_env/lib/python3.6/site-packages/scipy/integrate/odepack.py:248: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
(pid=81765)   warnings.warn(warning_msg, ODEintWarning)
(pid=81686) warning: basinhopping: local minimization failure
(pid=81686) basinhopping step 1: f -2.08615e+11 trial_f 1.67517e+08 accepted 0  lowest_f -2.08615e+11
(pid=81778) basinhopping step 1: f -1.2968e+11 trial_f 1.07105e+09 accepted 0  lowest_f -1.2968e+11
(pid=81726) basinhopping step 7: f -6.52627e+11 trial_f -5.10571e+11 accepted 0  lowest_f -6.52627e+11
(pid=81778) basinhopping step 2: f -1.2968e+11 trial_f 

(pid=81857) basinhopping step 1: f 4.06179e+07 trial_f 4.06179e+07 accepted 1  lowest_f 4.06179e+07
(pid=81857) found new global minimum on step 1 with function value 4.06179e+07
(pid=81792) basinhopping step 3: f -3.10741e+11 trial_f -3.10741e+11 accepted 1  lowest_f -3.10741e+11
(pid=81792) found new global minimum on step 3 with function value -3.10741e+11
(pid=81686) basinhopping step 8: f -2.08615e+11 trial_f 6.4868e+08 accepted 0  lowest_f -2.08615e+11
(pid=81844) basinhopping step 1: f -3.39526e+13 trial_f 8.09095e+09 accepted 0  lowest_f -3.39526e+13
(pid=81818) basinhopping step 4: f -2.61089e+11 trial_f 3.48242e+09 accepted 0  lowest_f -2.61089e+11
(pid=81793) basinhopping step 8: f -1.18759e+12 trial_f 9.13549e+08 accepted 0  lowest_f -1.18759e+12
(pid=81818) basinhopping step 5: f -2.61089e+11 trial_f 3.41348e+08 accepted 0  lowest_f -2.61089e+11
(pid=81686) basinhopping step 9: f -2.08615e+11 trial_f 2.54355e+08 accepted 0  lowest_f -2.08615e+11
(pid=81818) basinhopping st

(pid=81857) basinhopping step 10: f -1.66744e+11 trial_f -1.66744e+11 accepted 1  lowest_f -1.66744e+11
(pid=81857) found new global minimum on step 10 with function value -1.66744e+11
(pid=81844) warning: basinhopping: local minimization failure
(pid=81844) basinhopping step 8: f -7.57559e+13 trial_f -7.57559e+13 accepted 1  lowest_f -7.57559e+13
(pid=81844) found new global minimum on step 8 with function value -7.57559e+13
(pid=81900) basinhopping step 2: f -3.61273e+11 trial_f 6.10005e+08 accepted 0  lowest_f -3.61273e+11
(pid=81901) basinhopping step 5: f 1.24568e+08 trial_f 9.56116e+08 accepted 0  lowest_f 1.24568e+08
(pid=81844) basinhopping step 9: f -7.57559e+13 trial_f 2.45994e+09 accepted 0  lowest_f -7.57559e+13
(pid=81926) basinhopping step 2: f -2.27549e+11 trial_f 1.89122e+08 accepted 0  lowest_f -2.27549e+11
(pid=81831) basinhopping step 6: f -7.0291e+11 trial_f -2.77942e+11 accepted 0  lowest_f -7.0291e+11
(pid=81901) basinhopping step 6: f 1.24568e+08 trial_f 2.2274e+

(pid=81969) basinhopping step 3: f 1.59052e+08 trial_f 2.24219e+08 accepted 0  lowest_f 1.59052e+08
(pid=81994) basinhopping step 1: f 5.84916e+08 trial_f 1.09084e+09 accepted 0  lowest_f 5.84916e+08
(pid=81994) basinhopping step 2: f 5.84916e+08 trial_f 5.85271e+08 accepted 0  lowest_f 5.84916e+08
(pid=81968) warning: basinhopping: local minimization failure
(pid=81968) basinhopping step 2: f -7.74352e+11 trial_f -7.74352e+11 accepted 1  lowest_f -7.74352e+11
(pid=81968) found new global minimum on step 2 with function value -7.74352e+11
(pid=81940) basinhopping step 6: f -2.78167e+11 trial_f 6.02694e+07 accepted 0  lowest_f -2.78167e+11
(pid=82008) basinhopping step 0: f -2.3879e+11
(pid=82007) basinhopping step 0: f -1.64851e+12
(pid=81953) basinhopping step 8: f -8.38852e+13 trial_f 3.4761e+11 accepted 0  lowest_f -8.38852e+13
(pid=81968) basinhopping step 3: f -7.74352e+11 trial_f 1.32477e+09 accepted 0  lowest_f -7.74352e+11
(pid=82008) basinhopping step 1: f -5.55238e+11 trial_f

(pid=82173) basinhopping step 0: f 5.19278e+08


In [ ]:
for i in range(0,len(states)):    

    state=states[i]
    query = dfparam.query('state == "{}"'.format(state)).reset_index()
    parameters = np.array(query.iloc[:, 2:])[0]
    startdate,predict_range,s0,e0,a0,i0,r0,d0,startNCases,ratio,wcases,wrec = parameters
    Date = datetime.strptime(startdate, "%Y-%m-%d")
    end_date = Date + timedelta(days=+int(optimal[i][1]))
    dateStr=end_date.strftime("%Y-%m-%d")

    j = query['index'].values[0]
    dfparam.at[j, "s0"] = optimal[i][0]
    dfparam.at[j, "start-date"] = dateStr
    dfparam.at[j, "i0"] = optimal[i][2]
    dfparam.at[j, "WCASES"] = optimal[i][3]
    dfparam.at[j, "WREC"] = optimal[i][4]

    dfparam.to_csv("data/param_optimized_Yabox.csv", sep=",", index=False)
    dfparam
    